Goal of this Notebook:

'Start' and 'End' have multiple different formats. The goal is to put the information into a uniform and useable format.
This is done by splitting both columns up in 3 columns each. For days and months there is an indicator now that shows whether these values are given. The year is in an own column and it is 0 if there as none given. The original columns were deleted from the data.

# Imports

In [2]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re 
from math import isnan

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [31]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})

C:\Users\flori\AppData\Local\Temp\ipykernel_18324\1699356385.py:9: DtypeWarning: Columns (11,264) have mixed types. Specify dtype option on import or set low_memory=False.
  prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})


# Formatting

The following columns need to be formatted:
- start 
- end

In [32]:
data = prep.copy()
data.head()

,full_nr,name,ks,commentary,event_type,location,before_Christ,country_and_unit,participants_role,participant,parish,text,class,parameter,unit,value,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,collection_queue_nr,collection_additional_nr,element_count,legend,is_original,initial_info,damages,state,color,additional_text,type,source,city_municipality,country,material_Polish,material_RC Photo Paper,material_a pearl,material_acetate cellulose film,material_albumen paper,material_albumin paper,material_aluminium,material_amber,material_artificial fiber material,material_artificial leather,material_artificial material,material_atlas,material_ballpoint pen ink,material_birch,material_bone,material_brass,material_brocade (clothing variety),material_bronze,material_canvas,material_canvas (type of cloth),material_cardboard,material_cast iron,material_celluloid,material_ceramics,material_chalk,material_chamois leather,material_chamotte,material_charcoal,material_chromogen emulsion,material_chromogen paper,material_clay,material_clothing variety,material_collodion paper,material_colloid paper,material_colour,material_copper,material_cotton,material_crepe,material_crepe (cloth type),material_crystal,material_diffusion paper,material_email,material_emulsion,material_enamel paint,material_faience,material_feather,material_film,material_film (material),material_flint,material_from the bat,material_glass,material_gold,material_granite,material_graphite,material_gypsum,material_handmade paper,material_ink,material_iron,material_kalka,material_knitwear,material_leotard (type of clothing),material_linen,material_mascara,material_metal,material_metal fibers,material_moire (clothing variety),material_movie,material_nan,material_newsprint,material_nitrocellulose film,material_nut,material_oil paint,material_organic matter,material_paper,material_papier mache,material_photo emulsion,material_photo material,material_photo paper,material_photo plate,material_photographic material,material_plastic,material_plastic mass,material_plywood,material_porcelain,material_printing ink,material_quartz,material_rubber,material_salt paper,material_silk,material_silver,material_silver gelatin emulsion,material_silver gelatin paper,material_skin,material_slate,material_stone,material_synthetic fibers,material_synthetic material,material_tempera,material_textile,material_tin,material_trillion,material_watercolor paint,material_wax,material_white metal,material_wire,material_wood,material_wood material,material_wooden board,material_wool,material_yarn,technique_(close/together) sewing,technique_addition,technique_aquatint,technique_ballpoint pen,technique_binding techniques,technique_black and white photo,technique_black and white photography,technique_bronzing,technique_brushing,technique_burning,technique_carving,technique_casting,technique_chalk,technique_charcoal,technique_chromogen procedure,technique_collage,technique_collotype,technique_color photo,technique_color photography,technique_colored chalk,technique_colored pencil,technique_coloring,technique_copper engraving,technique_copying,technique_crayon drawing,technique_croaking,technique_crocheting,technique_cutting,technique_cuttlefish,technique_daguerreotype,technique_digital photography,technique_digital printing,technique_drawing,technique_electronic imaging,technique_embroidery,technique_enamelling,technique_engraving,technique_etching,technique_felt tip pen,technique_glass technology,technique_gouache,technique_graphics,technique_graphite,technique_handicraft,technique_handwriting,technique_ink,technique_ink drawing,technique_intarsia,technique_kiln ceramics,technique_knitting,technique_letterpress with raster cliché,technique_linocut,technique_lithography,technique_manuscript,technique_marker,technique_mascara,technique_mezzotinto,technique_mixed media,technique_modeling,technique_molding,technique_monotypy,technique_nan,technique_offset printing,te

In [25]:
#groups the year into its own column with the complete number, if NaN, then 0
def year_Grouping(x):
    xStr = str(x)
    if xStr == 'nan':
        return 0
    if '.' in xStr:
        xStr = xStr.split('.')
        xStr = xStr[xStr.__len__()-1]
        if xStr.__len__() == 2:
            xStr = '19' + xStr
        elif xStr == '':
            return 0
        return int(xStr)
    else:
        return int(xStr)

# returns 1 if a month is given, 0 if not
def month_Grouping(x):
    xStr = str(x)
    if '.' in xStr:
        xStr = xStr.split('.')
        if xStr[0].__len__() <= 2 and xStr[1].__len__() <= 2:
            if xStr[1] == '':
                return 0
            elif xStr[1].__len__() == 1:
                return 1
            return 1
        xStr = xStr[0]
        if xStr == 'jaan':
            return 1
        elif xStr == 'veebr':
            return 1
        if xStr == 'märts':
            return 1
        elif xStr == 'apr':
            return 1
        elif xStr == 'mai':
            return 1
        elif xStr == 'juuni':
            return 1
        elif xStr == 'juuli':
            return 1
        elif xStr == 'aug':
            return 1
        elif xStr == 'sept':
            return 1
        elif xStr == 'okt':
            return 1
        elif xStr == 'nov':
            return 1
        elif xStr == 'dets':
            return 1
        elif xStr.__len__() == 1:
            return 1
        return 1
    else:
        return 0

#returns one if a day is given, 0 if not
def day_Grouping(x):
    xStr = str(x)
    if '.' in xStr:
        xStr = xStr.split('.')
        if xStr[0].__len__() <= 2 and xStr[1].__len__() <= 2:
            return 1
        return 0
    else:
        return 0
    
    
#grouping applied to the dataframe
data['startYear'] = data['start'].apply(year_Grouping)
data['startMonth'] = data['start'].apply(month_Grouping)
data['startDay'] = data['start'].apply(day_Grouping)
#print(data['startYear'].unique())
print(data['startMonth'].unique())
print(data['startDay'].unique())
        
data['endYear'] = data['end'].apply(year_Grouping)
data['endMonth'] = data['end'].apply(month_Grouping)
data['endDay'] = data['end'].apply(day_Grouping)
#print(data['endYear'].unique())
#print(data['endMonth'].unique())
#print(data['endDay'].unique())


[0 1]
[0 1]


In [26]:
for i in range(1,len(data)):
    if data['startYear'].iloc[i] == 0 and data['startDay'].iloc[i] != 0:
        data['startYear'].iloc[i] = data['endYear'].iloc[i]
        

C:\Users\flori\AppData\Local\Temp\ipykernel_18324\3656531171.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['startYear'].iloc[i] = data['endYear'].iloc[i]


# Testing

In [28]:
#print(data[['start','startYear', 'startMonth', 'startDay','end', 'endYear', 'endMonth', 'endDay']])
data.drop(['start', 'end'], axis=1, inplace=True)

              start  startYear  startMonth  startDay         end  endYear  \
id                                                                          
232170         1979       1979           0         0         NaN        0   
2251378        1938       1938           0         0         NaN        0   
2070466         NaN          0           0         0         NaN        0   
4085096  18.09.1969       1969           1         1         NaN        0   
2697904         NaN          0           0         0         NaN        0   
2521183        1500       1500           0         0        1550     1550   
224392         1978       1978           0         0         NaN        0   
2111663         NaN          0           0         0         NaN        0   
2746349         NaN          0           0         0         NaN        0   
198694          NaN          0           0         0         NaN        0   
2748234      mai.44       1944           1         0         NaN        0   

In [29]:
#save 
data.to_csv('data/prep.csv')